In [1]:
!pip install qiskit qutip numpy matplotlib torch tqdm scikit-learn einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 73.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 89.7 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 26.9 MB/s  0:00:12m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 38.6 MB/s  0:00:08m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 42.2 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 107.1 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 63.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 31.7 MB/s  0:00:10m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 59.8 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.4 MB/s  0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 76.1 MB/s  0:00:006m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from qutip import basis, ket2dm, Qobj
import numpy as np

def mixed_state_generation(n_samples=10000):
    mixed_states = []
    for _ in range(n_samples):
        # Random angles on Bloch sphere
        theta = np.arccos(2*np.random.rand() - 1)  # polar angle [0, pi]
        phi = 2*np.pi*np.random.rand()             # azimuthal angle [0, 2pi]

        # Construct state |ψ> = cos(theta/2)|0> + exp(i*phi)*sin(theta/2)|1>
        psi = np.cos(theta/2)*basis(2,0) + np.exp(1j*phi)*np.sin(theta/2)*basis(2,1)
        pure_dm = ket2dm(psi)
        
        # Random mixing with |1> to get mixed state
        p = np.random.rand()
        rho_mixed = p * pure_dm + (1-p) * ket2dm(basis(2,1))
        mixed_states.append(rho_mixed)
            
    return mixed_states

mixed_state_data = mixed_state_generation()
print(mixed_state_data[0])

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.23863593+0.j         -0.32723432-0.12202914j]
 [-0.32723432+0.12202914j  0.76136407+0.j        ]]


In [6]:
from qutip import Qobj, ket2dm, tensor, ptrace
import numpy as np # Import numpy

def generate_clones(mixed_state_data):
  cloned_pairs = []
  U_clone = Qobj([[1,0,0,0],
                [0,1/np.sqrt(2),1/np.sqrt(2),0],
                [0,1/np.sqrt(2),-1/np.sqrt(2),0],
                [0,0,0,1]], dims=[[2, 2], [2, 2]]) # Corrected dimensions
  ancilla = ket2dm(basis(2,0))          # Blank qubit |0>
  for rho in mixed_state_data:  # mixed_states is your list of 10k arrays
    rho_joint = tensor(rho, ancilla)      # Combine system

    rho_cloned = U_clone * rho_joint * U_clone.dag()  # apply unitary

    clone1 = ptrace(rho_cloned, 0)
    clone2 = ptrace(rho_cloned, 1)
    cloned_pairs.append((clone1, clone2))
  return cloned_pairs

generated_clones = generate_clones(mixed_state_data)
print(generated_clones[0])

(Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.61931797+0.j         0.23138961+0.08628763j]
 [0.23138961-0.08628763j 0.38068203+0.j        ]], Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.61931797+0.j         -0.23138961-0.08628763j]
 [-0.23138961+0.08628763j  0.38068203+0.j        ]])


In [7]:
import numpy as np
from qutip import sigmax, sigmay, sigmaz, Qobj

def qobj_to_bloch(rho):
    """Convert a single QuTiP Qobj density matrix to a Bloch vector."""
    x = np.real((rho * sigmax()).tr())
    y = np.real((rho * sigmay()).tr())
    z = np.real((rho * sigmaz()).tr())
    return np.array([x, y, z])

def convert_pairs_to_bloch(pair_list):
    """
    Convert a list of (rho1, rho2) pairs to Bloch vectors.

    Returns two arrays:
      - bloch1: first clones
      - bloch2: second clones
    """
    bloch1 = []
    bloch2 = []

    for rho1, rho2 in pair_list:
        bloch1.append(qobj_to_bloch(rho1))
        bloch2.append(qobj_to_bloch(rho2))

    return np.array(bloch1), np.array(bloch2)

# Example usage
# pair_list = [(rho1_clone1, rho1_clone2), (rho2_clone1, rho2_clone2), ...]
bloch_clone1, bloch_clone2 = convert_pairs_to_bloch(generated_clones)

print(bloch_clone1.shape)  # (10000, 3)
print(bloch_clone2.shape)  # (10000, 3)


(10000, 3)
(10000, 3)


In [8]:
def convert_original_to_bloch(original_list):
    bloch = []

    for rho in original_list:
        bloch.append(qobj_to_bloch(rho))


    return np.array(bloch)

bloch_original = convert_original_to_bloch(mixed_state_data)

print(bloch_original.shape)  # (10000, 3)

(10000, 3)


In [9]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


x_train_orig, x_test_orig, x_train_c1, x_test_c1, x_train_c2, x_test_c2 = train_test_split(
    bloch_original, bloch_clone1, bloch_clone2, test_size=0.2, random_state=42
)

# Stack clones to create inputs and targets
X_train = np.vstack([x_train_c1, x_train_c2]).astype(np.float32)
y_train = np.vstack([x_train_orig, x_train_orig]).astype(np.float32)

X_test = np.vstack([x_test_c1, x_test_c2]).astype(np.float32)
y_test = np.vstack([x_test_orig, x_test_orig]).astype(np.float32)


# -------------------------------
# PyTorch Dataset
# -------------------------------
class BlochDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create datasets
train_dataset = BlochDataset(X_train, y_train)
test_dataset = BlochDataset(X_test, y_test)

# DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [10]:
import torch.nn as nn

class BlochDenoiser(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 3)
        )

    def forward(self, x):
        return self.net(x)

model = BlochDenoiser()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

n_epochs = 50

for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X_batch.size(0)
    avg_loss = total_loss / len(train_loader.dataset)

    # Evaluate on test set
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch)
            test_loss += criterion(output, y_batch).item() * X_batch.size(0)
    avg_test_loss = test_loss / len(test_loader.dataset)

    print(f"Epoch {epoch+1}/{n_epochs} | Train Loss: {avg_loss:.6f} | Test Loss: {avg_test_loss:.6f}")

Epoch 1/50 | Train Loss: 0.108738 | Test Loss: 0.077361
Epoch 2/50 | Train Loss: 0.075354 | Test Loss: 0.077330
Epoch 3/50 | Train Loss: 0.075184 | Test Loss: 0.077351
Epoch 4/50 | Train Loss: 0.075226 | Test Loss: 0.077308
Epoch 5/50 | Train Loss: 0.075128 | Test Loss: 0.077378
Epoch 6/50 | Train Loss: 0.075150 | Test Loss: 0.077207
Epoch 7/50 | Train Loss: 0.075197 | Test Loss: 0.077075
Epoch 8/50 | Train Loss: 0.075127 | Test Loss: 0.077141
Epoch 9/50 | Train Loss: 0.075179 | Test Loss: 0.077744
Epoch 10/50 | Train Loss: 0.075153 | Test Loss: 0.077195
Epoch 11/50 | Train Loss: 0.075187 | Test Loss: 0.077046
Epoch 12/50 | Train Loss: 0.075117 | Test Loss: 0.077209
Epoch 13/50 | Train Loss: 0.075227 | Test Loss: 0.077331
Epoch 14/50 | Train Loss: 0.075117 | Test Loss: 0.077234
Epoch 15/50 | Train Loss: 0.075087 | Test Loss: 0.077384
Epoch 16/50 | Train Loss: 0.075103 | Test Loss: 0.077725
Epoch 17/50 | Train Loss: 0.075166 | Test Loss: 0.077251
Epoch 18/50 | Train Loss: 0.075124 | Tes

In [11]:
import torch
import numpy as np

# Make sure the model is in evaluation mode
model.eval()

# Convert your test data to torch tensors if not already
X_test_tensor = torch.from_numpy(X_test)  # shape: (n_samples*2, 3)
y_test_tensor = torch.from_numpy(y_test)  # shape: (n_samples*2, 3)

# Run the model on test data
with torch.no_grad():
    y_pred = model(X_test_tensor)

# Convert predictions back to NumPy
y_pred_np = y_pred.numpy()


In [12]:
import numpy as np

def bloch_fidelity_batch(rho_array, sigma_array):
    """
    Compute fidelities between two arrays of Bloch vectors.

    Parameters:
        rho_array: np.array of shape (n_samples, 3) - true vectors
        sigma_array: np.array of shape (n_samples, 3) - predicted vectors

    Returns:
        fidelities: np.array of shape (n_samples,)
    """
    r_dot = np.sum(rho_array * sigma_array, axis=1)           # dot product for each sample
    r_norm = np.linalg.norm(rho_array, axis=1)               # magnitude of true vectors
    s_norm = np.linalg.norm(sigma_array, axis=1)             # magnitude of predicted vectors
    fidelities = 0.5 * (1 + r_dot + np.sqrt(1 - r_norm**2) * np.sqrt(1 - s_norm**2))
    return fidelities

# Example usage:
fidelities_clone_orig = bloch_fidelity_batch(y_test, X_test)
fidelities_pred_true = bloch_fidelity_batch(y_test, y_pred_np)

# Average fidelity over all test samples
print(f"Average pre trained fidelity: {np.mean(fidelities_clone_orig):.6f}")

# Average fidelity over all test samples
print(f"Average post trained fidelity: {np.mean(fidelities_pred_true):.6f}")

# Optional: inspect first few
for i in range(5):
    print(f"True: {y_test[i]}, Predicted: {y_pred_np[i]}, Fidelity: {fidelities_pred_true[i]:.6f}")


Average pre trained fidelity: 0.709880
Average post trained fidelity: 0.924395
True: [-0.53195876  0.40049288  0.03133399], Predicted: [ 0.02680653 -0.02383776  0.02983391], Fidelity: 0.860866
True: [-0.22185768  0.7300604  -0.49764797], Predicted: [ 0.0279863  -0.01694597 -0.49858037], Fidelity: 0.793418
True: [-0.24586426 -0.7821866   0.34952572], Predicted: [ 0.01299435 -0.01587716  0.3474653 ], Fidelity: 0.777858
True: [ 0.3833104  -0.14213307 -0.39633077], Predicted: [ 0.02699019 -0.00095847 -0.39650863], Fidelity: 0.960993
True: [0.6734333  0.28048235 0.5989049 ], Predicted: [-0.03570561  0.01990424  0.5775033 ], Fidelity: 0.798381


In [16]:
torch.save(model.state_dict(), "MLP.pth")


In [14]:
from google.cloud import storage

# Initialize a client
client = storage.Client()

# Reference your bucket
bucket_name = "instr-cs795-fall25-hqin-1-kseek001"   # example: "kris-models"
bucket = client.bucket(bucket_name)


In [17]:
!gsutil cp model.pth gs://instr-cs795-fall25-hqin-1-kseek001/torch_models/model.pth


Copying file://model.pth [Content-Type=application/octet-stream]...
/ [1 files][  4.2 KiB/  4.2 KiB]                                                
Operation completed over 1 objects/4.2 KiB.                                      
